In [5]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
from scipy.stats import poisson
import json
import sqlite3
import calendar
import datetime
from dateutil.relativedelta import relativedelta
import datetime
import os

SPREADSHEET_IDS = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
    "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
    "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    "ニュークラウン川越2号店": "1jocaH94-5GKWUqEqcrybgFcpb1Tn0BhvLMqaADppNWg",
    "TOHO川越店": "1VOr7BNMB-xnHihZrk6MFmetlZ0nCFP86YcrJu9w0dac",
}
# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [6]:
def create_df_from_database(hall_name, start_date, end_date, model_name=None):
    # Table name 取得
    DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        hall_id, hall_name = results[0]
        print(f"🔍 '{hall_name}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{hall_name}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,
            m.name AS model_name
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id
        JOIN models m ON r.model_id = m.model_id
        WHERE h.name = ?
        AND r.date BETWEEN ? AND ?
        """

    params = [hall_name, start_date, end_date]
    if model_name:
        query += " AND m.name LIKE ?"
        params.append(f"%{model_name}%")  # 部分一致にする

    query += " ORDER BY r.date DESC, r.unit_no ASC"

    df = pd.read_sql_query(query, conn, params=params)
    print(f"データサイズ: {df.shape[0]} x {df.shape[1]}")
    print(
        f"📅 検索期間: {start_date} ～ {end_date}",
        f"📅 抽出期間: {df.date.min()} ～ {df.date.max()}",
    )
    print(f'含まれる日数 : {df["date"].nunique()}')

    return df

In [7]:
# ブドウシミュレーター
# def grape_calc_myfive(game, bb, rb, medals, cherry=True):
#     bb_medals = 239.25
#     rb_medals = 95.25
#     replay_rate = 0.411
#     if cherry:
#         cherry_rate_high = 0.04228
#     else:
#         cherry_rate_high = 0.05847
#     denominator_inner = (
#         -medals
#         - (
#             game * 3
#             - (
#                 bb * bb_medals
#                 + rb * rb_medals
#                 + game * replay_rate
#                 + game * cherry_rate_high
#             )
#         )
#     ) / 8
#     grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
#     return grape_rate


GRAPE_CONSTANTS = {
    "マイジャグラーV": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.05847,
        "cherryOn": 0.04228,
    },
    "アイムジャグラーEX-TP": {
        "bb": 251.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.06068,
        "cherryOn": 0.040475,
    },
    "ゴーゴージャグラー3": {
        "bb": 239.00,
        "rb": 95.00,
        "replay": 0.411,
        "cherryOff": 0.0661,
        "cherryOn": 0.0372,
    },
    "ファンキージャグラー2": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.0603,
        "cherryOn": 0.04324,
    },
}


def calc_grape_rate(row, constants, cherry=True):
    model = row["model_name"]
    if model not in constants:
        return None
    try:
        game = row["game"]
        bb = row["BB"]
        rb = row["RB"]
        medals = row["medals"]
        # 定数取得
        c = constants[model]
        cherry_rate = c["cherryOn"] if cherry else c["cherryOff"]
        # 分母計算式
        denominator = (
            -medals
            - (
                game * 3
                - (
                    bb * c["bb"]
                    + rb * c["rb"]
                    + game * c["replay"]
                    + game * cherry_rate
                )
            )
        ) / 8
        if denominator == 0:
            return None  # ゼロ除算防止
        grape = (game / denominator) - ((game / denominator) * 2)
        return round(grape, 2)

    except Exception as e:
        print(f"⚠️ Grape計算失敗: {model} → {e}")
        return None


def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "other"


jagglar_rate_path = r"C:\python\dataOnline\anaslo_02\json\jagglar_rate.json"
with open(jagglar_rate_path, "r", encoding="utf-8") as f:
    MODEL_DATA_DICT = json.load(f)


# 設定推定関数
def estimate_setting_probs(row, limit_games=5000):
    # JSONファイルの確率表を読み込む
    model = row["model_name"]
    games = row["game"]
    if games < limit_games or pd.isna(row["RB_rate"]) or pd.isna(row["Total_rate"]):
        return {f"設定{i}": 0.0 for i in range(1, 7)}
    use_grape = not pd.isna(row["Grape_rate"]) and row["Grape_rate"] != 0
    rb_actual = row["RB"]
    total_actual = row["BB"] + row["RB"]
    if use_grape:
        grape_actual = round(games / row["Grape_rate"])
        
    jagglar_rate_path = r"C:\python\dataOnline\anaslo_02\json\jagglar_rate.json"
    with open(jagglar_rate_path, "r", encoding="utf-8") as f:
        MODEL_DATA_DICT = json.load(f)
    model_data = MODEL_DATA_DICT.get(model)
    if not model_data:
        return {i: 0.0 for i in range(1, 7)}

    likelihoods = {}
    for setting, values in model_data.items():
        try:
            rb_rate = float(values["RB_RATE"].split("/")[1])
            total_rate = float(values["TOTAL_RATE"].split("/")[1])
            grape_rate = float(values["GRAPE_RATE"].split("/")[1])
            expected_rb = games / rb_rate
            expected_total = games / total_rate
            expected_grape = games / grape_rate
            rb_likelihood = poisson.pmf(rb_actual, expected_rb)
            total_likelihood = poisson.pmf(total_actual, expected_total)
            # Grapeを使うかどうか
            if use_grape:
                grape_rate = float(values["GRAPE_RATE"].split("/")[1])
                expected_grape = games / grape_rate
                grape_likelihood = poisson.pmf(grape_actual, expected_grape)
                likelihood = rb_likelihood * total_likelihood * grape_likelihood
            else:
                likelihood = rb_likelihood * total_likelihood

            likelihoods[int(setting)] = likelihood
        except:
            continue
    total_sum = sum(likelihoods.values())
    if total_sum == 0:
        return {f"設定{i}": 0.0 for i in range(1, 7)}
    probs = {f"設定{s}": round(l / total_sum, 2) for s, l in likelihoods.items()}
    return probs


def df_preprocessing(df, hall_name):
    json_path = f"C:/python/dataOnline/anaslo_02/json/{hall_name}_area_map.json"
    if not os.path.exists(json_path):
        json_path = f"C:/python/dataOnline/anaslo_02/json/other_area_map.json"
    print(f"データ前処理を行います")
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre_columns = [
        "hall_name",
        "date",
        "model_name",
        "unit_no",
        "game",
        "BB",
        "RB",
        "medals",
    ]
    df_pre = df_pre[df_pre_columns]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = df_pre.apply(
        lambda row: calc_grape_rate(row, GRAPE_CONSTANTS), axis=1
    )
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)

    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]
    df_pre["last_digit"] = df_pre["date"].dt.day.astype(str).str[-1].astype(int)
    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, json_path))

    # 設定推定
    df_probs = df_pre.apply(estimate_setting_probs, axis=1, result_type="expand")
    df_pre["5more"] = df_probs[["設定5", "設定6"]].sum(axis=1)

    df_pre.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())

    return df_pre, model_list


def add_spreadsheet(spreadsheet, sheet_name, df, sheet_clear=False):
    today = datetime.datetime.today()
    rows, cols = df.shape
    try:
        sheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(
            title=sheet_name, rows=df.shape[0] + 3, cols=df.shape[1] + 3
        )
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    if sheet_clear:
        sheet.clear()
    last_row = len(sheet.get_all_values())
    set_with_dataframe(sheet, df, row=last_row, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

## ピボットテーブル作成


In [8]:
def create_pivot_table(
    df,
    index,
    columns,
    pivots=["game", "medals", "BB", "RB"],
    reverse=False,
    margins=True,
    day_target=None,
):
    df_filtered = df.copy()
    if day_target is not None:
        df_filtered = df_filtered[df_filtered["day"] == day_target]

    pivot_results = {}
    for col in pivots:
        table = df_filtered.pivot_table(
            index=index,
            columns=columns,
            values=col,
            aggfunc="sum",
            margins=margins,
            margins_name="total",
        )
        pivot_results[col] = table
        if reverse:
            pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]
    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)
    details = {
        "game": game,
        "medals": medals,
        "medal_rate": medal_rate,
        "bb": bb,
        "rb": rb,
        "rb_rate": rb_rate,
        "total_rate": total_rate,
    }

    return merged, details

## 出力データ

- 期間指定
  - 機種別 x 日付
  - 島別 x 日付
  - 台番号 x 日付
  - 島別 x 月
  - 過去 n 日の差枚比較


In [9]:
hall_name = "EXA FIRST"
# hall_name = "コンサートホールエフ成増"
# hall_name = "第一プラザみずほ台店"
# hall_name = "オータ志木駅前店"
# hall_name = "第一プラザ坂戸1000"
# hall_name = "ニュークラウン川越2号店"
# hall_name = "TOHO川越店"
# hall_name = "グランドオータ新座駅前店"
# hall_name = "ニューダイエイiii"

model_name = "ジャグラー"

In [10]:
today = datetime.date.today()
start_date = today - relativedelta(months=6, days=today.day - 1)
df_db = create_df_from_database(hall_name, start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
df.to_csv("df_preprocessing.csv", index=False, encoding="utf-8-sig")
spreadsheet = client.open_by_key(SPREADSHEET_IDS[hall_name])
print(f"スプレッドシート: {spreadsheet.title} を開きました。")
df.tail()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 31920 x 11
📅 検索期間: 2025-01-01 ～ 2025-07-31 📅 抽出期間: 2025-01-01 ～ 2025-07-30
含まれる日数 : 210
データ前処理を行います


KeyboardInterrupt: 

## MODEL_RATE_DAY


In [ ]:
sheet_name = "MODEL_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
model_rate = details["medal_rate"].copy()
model_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
model_name                                                                  
アイムジャグラーEX-TP      1.010  0.994  1.001  0.984  1.005  0.996  0.984  1.000   
ウルトラミラクルジャグラー      1.002  1.004  1.007  0.997  1.010  0.994  1.019  0.983   
ゴーゴージャグラー3         1.007  0.999  0.982  0.998  1.003  1.006  1.000  0.993   
ジャグラーガールズ          1.010  1.002  0.974  0.973  0.997  0.995  0.991  0.985   
ハッピージャグラーVIII      1.031  0.997  1.005  1.019  0.992  1.003  1.029  1.056   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
model_name                   ...                                             
アイムジャグラーEX-TP  0.980  1.000  ...  1.008  0.983  1.002  1.020  0.979  1.001   
ウルトラミラクルジャグラー  0.986  0.987  ...  0.988  0.986  1.004  1.030  0.998  0.981   
ゴーゴージャグラー3     0.995  1.007  ...  0.991  0.997  0.991  1.021  0.993  0.973   
ジャグラーガールズ      0.989  1.002  ...  0.990  0.990  1.008  0.994  0.986  0.986   
ハッピージャグラーVIII  0.985  1.002  ...  0.963  0.958  0.993  1.026  0.949  1.002   

                                           
day               29     30     31  total  
model_name                                 
アイムジャグラーEX-TP  0.997  1.011  0.993  0.999  
ウルトラミラクルジャグラー  0.990  0.985  0.975  0.997  
ゴーゴージャグラー3     0.986  1.000  0.989  0.997  
ジャグラーガールズ      0.980  0.984  0.987  0.995  
ハッピージャグラーVIII  0.964  0.904  1.011  0.996  

[5 rows x 32 columns]

## ISLAND_RATE


In [ ]:
sheet_name = f"ISLAND_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["area"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
island_rate = details["medal_rate"].copy()
island_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, island_rate, sheet_clear=True)
island_rate.head()

MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
area                                                                        
a: ゴーゴージャグラー3      1.031  0.989  1.003  1.002  1.002  1.014  1.000  0.975   
b: ゴーゴージャグラー3      1.031  0.990  0.992  1.013  1.003  1.021  1.001  0.993   
c: マイジャグラーV        1.057  0.981  0.998  1.019  0.996  0.997  1.005  0.994   
d: マイジャグラーV        1.010  0.983  1.008  0.980  1.017  0.996  0.989  1.002   
e: マイジャグラーV        1.032  1.019  1.003  0.982  1.029  1.003  0.996  1.010   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
area                         ...                                             
a: ゴーゴージャグラー3  1.006  1.023  ...  0.999  0.998  1.000  1.013  1.029  0.969   
b: ゴーゴージャグラー3  0.981  0.986  ...  1.006  1.022  1.009  1.009  1.018  0.975   
c: マイジャグラーV    1.007  0.994  ...  0.988  0.998  0.997  0.992  0.993  0.988   
d: マイジャグラーV    0.977  0.986  ...  0.974  0.988  1.020  1.023  1.003  0.998   
e: マイジャグラーV    0.985  1.006  ...  1.015  1.008  1.004  0.991  1.013  0.987   

                                           
day               29     30     31  total  
area                                       
a: ゴーゴージャグラー3  0.987  0.979  1.004  1.004  
b: ゴーゴージャグラー3  0.997  0.986  0.979  1.003  
c: マイジャグラーV    1.001  0.991  1.014  1.001  
d: マイジャグラーV    0.988  0.989  1.016  1.002  
e: マイジャグラーV    0.990  0.999  1.018  1.005  

[5 rows x 32 columns]

## UNIT_RATE


In [ ]:
sheet_name = f"UNIT_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name", "unit_no"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
target_rate = 1.05
unit_rate = details["medal_rate"].copy()
unit_rate[("MEDAL_RATE", f"{target_rate}+")] = (
    unit_rate.iloc[:, :-1] >= target_rate
).sum(axis=1)
countif = (unit_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
unit_rate = pd.concat(
    [unit_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0
)
unit_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
unit_rate = unit_rate[~unit_rate.iloc[:, 1].isna()]
unit_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, unit_rate, sheet_clear=True)
unit_rate.head()

MEDAL_RATE                                            \
day                         1      2      3      4      5      6      7   
アイムジャグラーEX-TP 1100      1.018  0.939  0.980  0.979  1.039  1.105  0.783   
              1101      1.027  0.941  1.084  0.936  1.007  0.900  0.922   
              1102      1.029  1.060  1.009  0.924  0.945  1.066  0.861   
              1103      1.033  1.153  1.010  0.819  1.078  1.059  0.876   
              1104      1.019  1.015  0.967  1.029  0.960  1.124  0.740   

                                         ...                              \
day                     8      9     10  ...     24     25     26     27   
アイムジャグラーEX-TP 1100  0.809  0.705  1.019  ...  0.977  0.997  0.957  1.016   
              1101  0.948  0.862  1.081  ...  0.826  0.880  1.019  1.143   
              1102  0.862  0.902  1.109  ...  0.917  1.010  1.086  0.960   
              1103  0.974  0.710  0.821  ...  0.951  0.968  1.002  0.953   
              1104  0.903  0.722  0.931  ...  1.012  0.910  0.991  0.988   

                                                             
day                    28     29     30     31  total 1.05+  
アイムジャグラーEX-TP 1100  1.069  1.017  1.059  1.016  0.998     4  
              1101  1.075  0.989  0.924  0.849  1.017     8  
              1102  0.912  0.987  1.007  1.008  1.013     7  
              1103  1.116  0.921  0.992  1.063  1.008     8  
              1104  1.068  1.001  0.900  0.891  0.996     4  

[5 rows x 33 columns]

## LAST_DIGIT_RATE


In [ ]:
sheet_name = f"LAST_DIGIT_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["last_digit"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
last_digit_rate = details["medal_rate"]
last_digit_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
last_digit_rate = last_digit_rate[~last_digit_rate.iloc[:, 1].isna()]
last_digit_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, last_digit_rate, sheet_clear=True)

# DAYn

- SHEET_NAME : DAY1，DAY2，… DAY31


In [ ]:
index = ["area", "unit_no"]
columns = ["date"]
day_list = [21]
# for day_target in range(today.day - 1, today.day + 1):
for day_target in day_list:
    sheet_name = f"DAY{day_target}"
    csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
    merged, details = create_pivot_table(
        df, index, columns, reverse=True, margins=True, day_target=day_target
    )
    merged.to_csv(csv_path)
    # add_spreadsheet(spreadsheet, f"DAY{day_target}", merged, sheet_clear=True,)
merged.head()

GAME MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  \
date                   total  total total total      total   total      total   
area          unit_no                                                           
a: ゴーゴージャグラー3 1001     21093  -3757    68    52      0.941   405.6      175.8   
              1002     29143   2659   122    80      1.030   364.3      144.3   
              1003     34857    872   142   100      1.008   348.6      144.0   
              1004     24311  -1351    86    72      0.981   337.7      153.9   
              1005     30733   5480   134    95      1.059   323.5      134.2   

                                     GAME              MEDALS  \
date                  2025-05-21 00:00:00 2025-05-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                   5013                -768   
              1002                   2270                -209   
              1003                   8640                 -88   
              1004                   1857                 232   
              1005                   4248                -332   

                                       BB  ...          MEDAL_RATE  \
date                  2025-05-21 00:00:00  ... 2025-01-21 00:00:00   
area          unit_no                      ...                       
a: ゴーゴージャグラー3 1001                     16  ...               0.790   
              1002                      8  ...               1.092   
              1003                     30  ...               0.881   
              1004                      9  ...               0.976   
              1005                     15  ...               1.108   

                                  RB_RATE          TOTAL_RATE  \
date                  2025-01-21 00:00:00 2025-01-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  525.0               300.0   
              1002                  326.1               128.9   
              1003                  592.0               197.3   
              1004                  351.9               159.9   
              1005                  292.6               122.3   

                                     GAME              MEDALS  \
date                  2024-12-21 00:00:00 2024-12-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                   5358                  76   
              1002                   3329                -432   
              1003                   7335                 171   
              1004                   2232               -1053   
              1005                   8782                2144   

                                       BB                  RB  \
date                  2024-12-21 00:00:00 2024-12-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                     21                  12   
              1002                     11                   7   
              1003                     30                  19   
              1004                      4                   5   
              1005                     39                  30   

                               MEDAL_RATE             RB_RATE  \
date                  2024-12-21 00:00:00 2024-12-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  1.005               446.5   
              1002                  0.957               475.6   
              1003                  1.008               386.1   
              1004                  0.843               446.4   
              1005                  1.081               292.7   

                               TOTAL_RATE  
date                  2024-12-21 00:00:00  
area          unit_no                      
a: ゴーゴージャグラー3 1001                  162.4  
              1002               

## MONTH_RATE


In [ ]:
start_date = today - relativedelta(months=100, days=today.day - 1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
sheet_name = "MONTH_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["month"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
month_rate = details["medal_rate"].copy()
# add_spreadsheet(spreadsheet, sheet_name, month_rate, sheet_clear=True)
month_rate.to_csv(csv_path)
month_rate

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 65208 x 11
📅 検索期間: 2017-02-01 ～ 2025-06-06 📅 抽出期間: 2024-04-01 ～ 2025-06-05
含まれる日数 : 429
データ前処理を行います


MEDAL_RATE                                                  \
month              total 2025-06 2025-05 2025-04 2025-03 2025-02 2025-01   
model_name                                                                 
アイムジャグラーEX-TP      1.002   1.014   0.998   0.998   1.001   0.993   0.997   
ウルトラミラクルジャグラー      0.997   0.981   0.989   0.993   0.997   1.002   1.000   
ゴーゴージャグラー3         1.002   0.999   0.999   0.993   0.992   1.001   1.000   
ジャグラーガールズ          1.003   1.005   0.994   0.993   1.001   0.990   0.996   
ハッピージャグラーVIII      0.996   1.027   0.990   0.995     NaN     NaN     NaN   
ファンキージャグラー2        0.996   0.980   0.992   0.992   0.993   0.996   0.990   
マイジャグラーV           1.010   1.010   1.003   1.000   1.002   1.003   1.006   
ミスタージャグラー          1.001   1.007   0.997   1.002   0.991   0.996   0.990   
total              1.006   1.006   0.999   0.998   0.999   1.001   1.002   

                                                                               \
month         2024-12 2024-11 2024-10 2024-09 2024-08 2024-07 2024-06 2024-05   
model_name                                                                      
アイムジャグラーEX-TP   1.000   1.001   0.999   1.001   1.000   1.007   1.005   1.004   
ウルトラミラクルジャグラー   1.001     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ゴーゴージャグラー3      0.996   1.005   0.995   0.998   1.003   1.003   1.006   1.010   
ジャグラーガールズ       0.994   0.995   0.999   1.003   0.997   1.005   1.005   1.010   
ハッピージャグラーVIII     NaN   0.957   1.004     NaN     NaN     NaN     NaN     NaN   
ファンキージャグラー2     0.995   0.996   1.000   0.989   1.001   0.986   0.990   1.000   
マイジャグラーV        1.005   1.002   1.008   1.009   1.012   1.011   1.017   1.021   
ミスタージャグラー       1.006   0.997   1.002   1.008   1.007   1.009     NaN     NaN   
total           1.002   1.001   1.004   1.005   1.007   1.008   1.011   1.015   

                       
month         2024-04  
model_name             
アイムジャグラーEX-TP   1.008  
ウルトラミラクルジャグラー     NaN  
ゴーゴージャグラー3      1.012  
ジャグラーガールズ       1.016  
ハッピージャグラーVIII     NaN  
ファンキージャグラー2     1.007  
マイジャグラーV        1.025  
ミスタージャグラー         NaN  
total           1.018

## HISTORY


In [ ]:
start_date = datetime.date(today.year, today.month - 1, 1) - relativedelta(days=6)
df_db = create_df_from_database(hall_name, start_date, today, model_name="ジャグラー")
df, model_list = df_preprocessing(df_db, hall_name)


pivot_targets = ["medals", "game", "RB_rate", "Total_rate", "Grape_rate"]
index_targets = ["area", "model_name", "unit_no"]
columns_targets = ["date"]

merged_by_model = pd.DataFrame()
for model in df["model_name"].unique():

    # モデルごとにデータをフィルタリング
    df_model = df[df["model_name"] == model]
    pivot_results = {}
    for col in pivot_targets:
        table = df_model.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
        )
        pivot_results[col] = table

    medals = pivot_results["medals"]
    game = pivot_results["game"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    grape_rate = pivot_results["Grape_rate"]
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    # ランキング列作成
    rolling7 = medals.T.rolling(7, min_periods=7).sum().T
    medal_rank7 = rolling7.rank(method="min", ascending=True)
    medal_rank7 = medal_rank7.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling5 = medals.T.rolling(5, min_periods=5).sum().T
    medal_rank5 = rolling5.rank(method="min", ascending=True)
    medal_rank5 = medal_rank5.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling3 = medals.T.rolling(3, min_periods=3).sum().T
    medal_rank3 = rolling3.rank(method="min", ascending=True)
    medal_rank3 = medal_rank3.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    medal_rank1 = medals.rank(method="min", ascending=True)
    medal_rank1 = medal_rank1.fillna(0).replace([np.inf, -np.inf], 0).astype(int)

    # MultiIndex化（ラベル付け）
    labeled_tables = [
        ("GRAPE_RATE", grape_rate),
        ("TOTAL_RATE", total_rate),
        ("RB_RATE", rb_rate),
        ("GAME", game),
        ("RATE_MEDAL", medal_rate),
        ("MEDALS", medals),
        ("3ROLLING", rolling3),
        ("5ROLLING", rolling5),
        ("7ROLLING", rolling7),
        ("1RANK", medal_rank1),
        ("3RANK", medal_rank3),
        ("5RANK", medal_rank5),
        ("7RANK", medal_rank7),
    ]
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去・日付ソート・一部データ削除
    interleaved_cols = [
        col
        for col_group in zip(*(df.columns for _, df in labeled_tables))
        for col in col_group
    ]
    merged = pd.concat([df for _, df in labeled_tables], axis=1)[interleaved_cols]
    merged = merged.iloc[:, ::-1]
    # display(merged)
    merged = merged[~merged.iloc[:, 7].isna()]

    # エリアごとに空行挿入して整形
    merged_by_area = pd.DataFrame()
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        if not area_merged.empty:
            empty_index = pd.MultiIndex.from_tuples(
                [("", " ", " ")], names=merged.index.names
            )
            empty_row = pd.DataFrame(
                [[""] * area_merged.shape[1]],
                index=empty_index,
                columns=area_merged.columns,
            )
            merged_by_area = pd.concat([merged_by_area, area_merged, empty_row])

    merged_by_area.to_csv("test.csv")

    # インデックス削除
    merged_by_area = merged_by_area.droplevel("area")

    if not merged_by_area.empty:
        empty_index = pd.MultiIndex.from_tuples(
            [(" ", " ")], names=merged_by_area.index.names
        )
        empty_row = pd.DataFrame(
            [[""] * merged_by_area.shape[1]],
            index=empty_index,
            columns=merged_by_area.columns,
        )
        merged_by_model = pd.concat(
            [merged_by_model, merged_by_area, empty_row], axis=0
        )

# 累計した最後の7日を削除
merged_by_model = merged_by_model.iloc[:, : -len(labeled_tables) * 6]
merged_by_model.replace([np.inf, -np.inf], np.nan, inplace=True)

sheet_name = "HISTORY"

csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
merged_by_model.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, merged_by_model, sheet_clear=True)

merged_by_model.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 6384 x 11
📅 検索期間: 2025-04-25 ～ 2025-06-06 📅 抽出期間: 2025-04-25 ～ 2025-06-05
含まれる日数 : 42
データ前処理を行います


7RANK      5RANK      3RANK      1RANK   7ROLLING  \
date               2025-06-05 2025-06-05 2025-06-05 2025-06-05 2025-06-05   
model_name unit_no                                                          
ゴーゴージャグラー3 1001             6          6          7          1    -1479.0   
           1002            11          8         11         15      661.0   
           1003             9         15         19         20       58.0   
           1004             7          2          2          5    -1010.0   
           1005            13          9         12         10      924.0   

                     5ROLLING   3ROLLING     MEDALS RATE_MEDAL       GAME  \
date               2025-06-05 2025-06-05 2025-06-05 2025-06-05 2025-06-05   
model_name unit_no                                                          
ゴーゴージャグラー3 1001       -1541.0     -756.0      -2568      0.795       4172   
           1002       -1203.0     -450.0         79      1.004       6433   
           1003         276.0     1979.0       2444       1.12       6784   
           1004       -3730.0    -2059.0       -982      0.878       2681   
           1005        -773.0     -438.0       -685      0.782       1049   

                    ...      1RANK   7ROLLING   5ROLLING   3ROLLING  \
date                ... 2025-05-01 2025-05-01 2025-05-01 2025-05-01   
model_name unit_no  ...                                               
ゴーゴージャグラー3 1001     ...          3     -618.0    -3997.0    -1070.0   
           1002     ...         20      755.0      191.0     1038.0   
           1003     ...         15      -44.0     1006.0       -3.0   
           1004     ...          6     -636.0    -1680.0    -1180.0   
           1005     ...         13     1449.0      543.0    -1787.0   

                       MEDALS RATE_MEDAL       GAME    RB_RATE TOTAL_RATE  \
date               2025-05-01 2025-05-01 2025-05-01 2025-05-01 2025-05-01   
model_name unit_no                                                          
ゴーゴージャグラー3 1001         -1279      0.909       4672      584.0      186.9   
           1002          1862      1.079       7842      201.1      110.5   
           1003          1253      1.056       7421      239.4      119.7   
           1004          -347      0.975       4573      304.9      142.9   
           1005           574      1.026       7349      306.2      136.1   

                   GRAPE_RATE  
date               2025-05-01  
model_name unit_no             
ゴーゴージャグラー3 1001          6.42  
           1002          5.96  
           1003          6.04  
           1004          6.27  
           1005          5.95  

[5 rows x 468 columns]

## HISTORY 島別の合算を計算追加


In [1]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
from scipy.stats import poisson
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import os





def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "other"


def df_db(hall_name, search_word, end_date, start_date):
    DB_PATH = r"C:/python/dataOnline/anaslo_02/db/anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
    )
    query = """
    -- 出玉データにホール名と機種名を結合して取得
    SELECT
        r.*, 
        h.name AS hall_name,
        m.name AS model_name
    FROM results r
    JOIN halls h ON r.hall_id = h.hall_id
    JOIN models m ON r.model_id = m.model_id
    WHERE h.name = ?
    AND r.date BETWEEN ? AND ?
    """
    params = [hall_name, start_date, end_date]
    if search_word:
        query += " AND m.name LIKE ?"
        params.append(f"%{search_word}%")
    query += " ORDER BY r.date DESC, r.unit_no ASC"
    df_db = pd.read_sql_query(query, conn, params=params)

    json_path = f"C:/python/dataOnline/anaslo_02/json/{hall_name}_area_map.json"
    # df_db = df_db.copy()
    df_db["date"] = pd.to_datetime(df_db["date"])
    df_db["area"] = df_db["unit_no"].apply(lambda x: assign_area(x, json_path))
    area_list = df_db["area"].unique()
    conn.close()
    
    return df_db, area_list




def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "other"


def make_df_pivot(df_db, area):
    df_area = df_db[df_db["area"] == area]
    model_name = df_area["model_name"].unique()[0]
    df_pivot = df_area.pivot_table(
        values=["medals", "game", "BB", "RB"],
        index=["area", "model_name", "unit_no"],
        columns="date",
        aggfunc="sum",
        margins=True,
        margins_name="sum",
    )
    return model_name, df_pivot





def estimate_grape_rate(model_name, df_pivot):
    jagglar_rate_path = r"C:\python\dataOnline\anaslo_02\json\jagglar_rate.json"
    with open(jagglar_rate_path, "r", encoding="utf-8") as f:
        MODEL_DATA_DICT = json.load(f)
    # c = MODEL_DATA_DICT[model_name]["grape_contents"]
    # c = MODEL_DATA_DICT.get(model_name, {}).get("grape_contents", None)
    DEFAULT_GRAPE_CONSTANTS = {
        "bb": 240,"rb": 95,"replay": 0.4,"cherryOff": 0.06,"cherryOn": 0.04
    }
    c = MODEL_DATA_DICT.get(model_name, {}).get("grape_contents", DEFAULT_GRAPE_CONSTANTS)
    # medal_out = -df_pivot["medals"]
    total_in_medals = df_pivot["game"] * 3
    bb_medals = df_pivot["BB"] * c["bb"]
    rb_medals = df_pivot["RB"] * c["rb"]
    replay_medals = df_pivot["game"] * c["replay"]
    cherry_medals = df_pivot["game"] * c["cherryOff"]
    grape_hits = (
        -1
        * (
            -df_pivot["medals"]
            - (total_in_medals - bb_medals - rb_medals - replay_medals - cherry_medals)
        )
        / 8
    )
    estimated_grape_rate = (df_pivot["game"] / grape_hits).round(2)
    return estimated_grape_rate


def make_df_interleaved(df_pivot, model_name, estimate_grape_rate):
    rb_rate = (df_pivot["game"] / df_pivot["RB"]).round(1)
    total_rate = (df_pivot["game"] / (df_pivot["BB"] + df_pivot["RB"])).round(1)
    medal_rate = ((df_pivot["medals"] + df_pivot["game"] * 3) / (df_pivot["game"] * 3)).round(3)
    grape_rate = estimate_grape_rate(model_name, df_pivot)
    rolling7 = df_pivot["medals"].T.rolling(7, min_periods=7).sum().T
    medal_rank7 = rolling7.iloc[:-1, :].rank(method="min", ascending=True)
    medal_rank7 = medal_rank7.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    game_rank = df_pivot["game"].rank(method="min", ascending=True)

    # 設定予測
    jagglar_rate_path = r"C:\python\dataOnline\anaslo_02\json\jagglar_rate.json"
    with open(jagglar_rate_path, "r", encoding="utf-8") as f:
        MODEL_DATA_DICT = json.load(f)
    model_data = MODEL_DATA_DICT[model_name]
    if not model_data:
        model_data = {str(i): {} for i in range(1, 7)}

    likelihoods = {}
    epsilon = 1e-10

    # 設定ごとに log likelihood を計算
    for setting, values in model_data.items():
        if setting in map(str, range(1, 7)):
            # if model_data[setting] == {}:
            #     model_data[setting] = {
            #         'BB_RATE': '1/2', 'RB_RATE': '1/2', 'GRAPE_RATE': '1/5', 'TOTAL_RATE': '1/100'}
            # 各設定のモデル確率（分母部分）を取得
            model_rb_rate = float(model_data[setting]["RB_RATE"].split("/")[1])
            model_total_rate = float(model_data[setting]["TOTAL_RATE"].split("/")[1])
            model_grape_rate = float(model_data[setting]["GRAPE_RATE"].split("/")[1])

            # 期待値を計算
            expected_rb = df_pivot["game"] / model_rb_rate
            expected_total = df_pivot["game"] / model_total_rate
            expected_grape = df_pivot["game"] / model_grape_rate

            # 各項目の PMF (尤度の要素) を計算
            rb_likelihood = poisson.pmf(df_pivot["RB"], expected_rb)
            total_likelihood = poisson.pmf(
                (df_pivot["BB"] + df_pivot["RB"]), expected_total
            )
            # ブドウ回数（実測値）を整数に丸めてPMF計算
            grape_actual = round(df_pivot["game"] / grape_rate)
            grape_likelihood = poisson.pmf(grape_actual, expected_grape)

            # 総合尤度（台×日付ごと）
            likelihood = rb_likelihood * total_likelihood * grape_likelihood

            # log likelihood に変換して格納（計算の安定性のために log で保持）
            log_likelihood = np.log(likelihood + epsilon)
            likelihoods[int(setting)] = log_likelihood

    # マスク作成
    game_mask = (df_pivot["game"] >= 5000).astype(float)
    # 設定ごとの likelihood 配列に変換 → (設定数, 台数, 日付数)
    likelihood_array = np.stack(
        [np.exp(likelihoods[s]) for s in sorted(likelihoods.keys())], axis=0
    )
    # ゲーム数5000以下は likelihood を 0 にする
    likelihood_array *= game_mask.values[np.newaxis, :, :]  # ブロードキャスト適用
    # 台×日付ごとに正規化 → 設定ごとの確率を算出
    likelihood_sum = likelihood_array.sum(
        axis=0, keepdims=True
    )  # shape: (1, 台数, 日付数)
    with np.errstate(divide="ignore", invalid="ignore"):
        likelihood_probs = np.divide(
            likelihood_array,
            likelihood_sum,
            out=np.zeros_like(likelihood_array),
            where=likelihood_sum != 0,
        )

    selected_settings = [5, 6]  # 足したい設定番号（4,5,6 など）
    prob_sum = sum(
        pd.DataFrame(
            likelihood_probs[s - 1],  # 0-indexなので s-1
            columns=df_pivot["game"].columns,
            index=df_pivot["game"].index,
        )
        for s in selected_settings
    ).round(2)
    # display(prob_sum)

    labeled_tables = [
        ("5MORE", prob_sum),
        # ("GRAPE_RATE", grape_rate),
        # ("TOTAL_RATE", total_rate),
        # ("RB_RATE", rb_rate),
        # ("RB", df_pivot["RB"]),
        # ("BB", df_pivot["BB"]),
        # ("GAME", df_pivot["game"]),
        # ("RATE_MEDALS", medal_rate),
        # ("MEDALS", df_pivot["medals"]),
        # ("7ROLLING", rolling7),
        # ("7RANK", medal_rank7),
        # ("GAME_RANK", game_rank),
    ]
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    interleaved_cols = [
        col
        for col_group in zip(*(df.columns for _, df in labeled_tables))
        for col in col_group
    ]

    df_interleaved = pd.concat([df for _, df in labeled_tables], axis=1)[
        interleaved_cols
    ]
    df_interleaved = df_interleaved.iloc[:, ::-1]
    df_interleaved = df_interleaved.iloc[:, len(labeled_tables) :]
    df_interleaved = df_interleaved.droplevel("area")
    df_interleaved.replace([np.inf, -np.inf, np.nan], None, inplace=True)
    
    return df_interleaved, rolling7





def add_spreadsheet(sheet_name, hall_name, df_interleaved):
    spreadsheet_ids = {
        "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
        "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
        "ニュークラウン川越2号店": "1jocaH94-5GKWUqEqcrybgFcpb1Tn0BhvLMqaADppNWg",
        "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
        "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
        "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
        "みずほ台uno": "1PhH3DbHjUTmsE0yJ7U85KKeFwqtSD7x7VZvTD7ZQO74",
        "第一プラザ坂戸にっさい店": "1IZrivGLeE30LgT1VvHhqcgLkOMcmdy-4GIC-4KEchbE",
        "オータ志木駅前店": "1d5UhcseEl7jscVqEk-u3-DZgi5EmIFnMhd6jfrPbpzw",
        "グランドオータ新座駅前店": "1cPAH5IW_qlViRKCc0eR1m-NN8eBuJ_0sjrDDYT_DWas",
        "toho川越店": "1VOr7BNMB-xnHihZrk6MFmetlZ0nCFP86YcrJu9w0dac",
        "ニューダイエイiii": "1CBDcxHjGT11_ujn0v_m-xDIZKJ6Xw1RaFzGdOt3jFVo",
        "レイト平塚": "1WlvgMk-u_tUWizQVOYDQVg7bECyVzLm0-wlJDgJteAw",
        "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
        "楽園大山店": "1dewRwbCe783UAj7C_r5tn4CA7JyPfwxCahc8aOf9M2I",
        "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
        "スロットエランドール田無店": "1czMHMqEjt9__Ou6tyKnWxhFa6atfhjeSDaFbf1Ef-c8",
        }
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive",
    ]
    jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
    creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    client = gspread.authorize(creds)
    spreadsheet = client.open_by_key(spreadsheet_ids[hall_name])
    rows, cols = df_interleaved.shape[0] + 5, df_interleaved.shape[1] + 4
    try:
        sheet = spreadsheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=sheet_name, rows=rows, cols=cols)

    last_row = len(sheet.get_all_values())
    if last_row == 1:
        set_with_dataframe(
            sheet,
            df_interleaved,
            row=last_row,
            include_index=True,
            include_column_header=True,
        )
    else:
        set_with_dataframe(
            sheet,
            df_interleaved,
            row=last_row + 2,
            include_index=True,
            include_column_header=False,
        )
    # today = datetime.date.today()
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d"))
    sheet.update_cell(1, 2, today.strftime("UPDATED"))
    
    
search_word = "ジャグラー"

sheet_name = "MORE5"
sheet_name = "SUMMER2024"

hall_name = "EXA FIRST"
# hall_name = "ニューダイエイiii"
hall_name = "楽園大山店"
# hall_name = "パールショップともえ川越店"
# hall_name = "コンサートホールエフ成増"
# hall_name = "スロットエランドール田無店"

today = datetime.date.today()
end_date = today - relativedelta(days=345)
start_date = today - relativedelta(days=360)
df_db, area_list = df_db(hall_name, search_word, end_date, start_date)
area_list = sorted(area_list)
# print(area_list)

for i, area in enumerate(area_list):
    # if i == 1: break
    model_name, df_pivot = make_df_pivot(df_db, area)
    df_interleaved, rolling7 = make_df_interleaved(df_pivot, model_name, estimate_grape_rate)
    # display(df_interleaved.iloc[:, :11])
    add_spreadsheet(sheet_name, hall_name, df_interleaved)

In [ ]:
today - datetime.date(2025, 5, 13)

datetime.timedelta(days=58)

In [ ]:
start_date = datetime.date(today.year, today.month - 1, 1) - relativedelta(days=6)
df_db = create_df_from_database(hall_name, start_date, today, model_name="ジャグラー")
df, model_list = df_preprocessing(df_db, hall_name)
df["last_digit"].head()

sheet_name = "SETTING_PREDICT"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
df = df[(df["last_digit"] == 1) | (df["last_digit"] == 5)]
index = ["model_name", "unit_no"]
columns = ["date"]
df_pred = df.pivot_table(
    index=index,
    columns=columns,
    values=["5more"],
    # aggfunc="sum",
    # margins=True,
    # margins_name="total",
)
df_pred = df_pred.iloc[:, ::-1]
df_pred = df_pred[~df_pred.iloc[:, 1].isna()]
df_pred.to_csv(csv_path)
df_pred

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 6688 x 11
📅 検索期間: 2025-04-25 ～ 2025-06-07 📅 抽出期間: 2025-04-25 ～ 2025-06-07
含まれる日数 : 44
データ前処理を行います


5more                                              \
date                  2025-06-05 2025-06-01 2025-05-31 2025-05-25 2025-05-21   
model_name    unit_no                                                          
アイムジャグラーEX-TP 1100          0.00       0.83       0.00       0.01       0.00   
              1101          0.42       0.55       0.00       0.00       0.00   
              1102          0.00       0.04       0.34       0.28       0.00   
              1103          0.00       0.84       0.45       0.00       0.37   
              1104          0.00       0.20       0.00       0.19       0.00   
...                          ...        ...        ...        ...        ...   
ミスタージャグラー     1109          0.00       0.00       0.00       0.00       0.00   
              1110          0.00       0.00       0.00       0.00       0.00   
              1111          0.00       0.00       0.00       0.00       0.00   
              1112          0.00       0.00       0.00       0.00       0.00   
              1113          0.00       0.00       0.00       0.00       0.00   

                                                                              
date                  2025-05-15 2025-05-11 2025-05-05 2025-05-01 2025-04-25  
model_name    unit_no                                                         
アイムジャグラーEX-TP 1100          0.30       0.02       0.92       0.01       0.22  
              1101          0.00       0.94       0.01       0.00       0.00  
              1102          0.08       0.99       0.00       0.02       0.00  
              1103          0.00       0.15       0.06       0.10       0.24  
              1104          0.00       0.75       0.05       0.85       0.00  
...                          ...        ...        ...        ...        ...  
ミスタージャグラー     1109          0.00       0.00       0.00       0.00       0.00  
              1110          0.00       0.00       0.00       0.00       0.00  
              1111          0.00       0.00       0.00       0.00       0.00  
              1112          0.00       0.00       0.00       0.00       0.00  
              1113          0.00       0.00       0.00       0.00       0.00  

[152 rows x 10 columns]

## MODEL_RATE_HISTORY


In [ ]:
last_month = today - relativedelta(months=0)
month_ago = today - relativedelta(months=1)
last_day = calendar.monthrange(last_month.year, last_month.month)[1]
end_date = datetime.date(last_month.year, last_month.month, last_day)
last_day = calendar.monthrange(month_ago.year, month_ago.month)[1]
start_date = datetime.date(month_ago.year, month_ago.month, 1)

df_db = create_df_from_database(
    hall_name, start_date, end_date, model_name="ジャグラー"
)
df, model_list = df_preprocessing(df_db, hall_name)
index = ["model_name", "unit_no"]
columns = ["date"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
model_rate = details["medal_rate"].copy()
model_rate = model_rate[~model_rate.iloc[:, 2].isna()]
model_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
model_rate.to_csv(csv_path)
sheet_name = "MODEL_RATE_HISTORY"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
# add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 5472 x 11
📅 検索期間: 2025-05-01 ～ 2025-06-30 📅 抽出期間: 2025-05-01 ～ 2025-06-05
含まれる日数 : 36
データ前処理を行います


MEDAL_RATE                                          \
date                       total 2025-06-05 00:00:00 2025-06-04 00:00:00   
model_name    unit_no                                                      
アイムジャグラーEX-TP 1100         0.986               0.883               1.039   
              1101         1.019               1.046               0.971   
              1102         1.023               0.855               0.955   
              1103         1.017               1.151               0.891   
              1104         1.004               1.039               0.958   

                                                               \
date                  2025-06-03 00:00:00 2025-06-02 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  0.957               0.920   
              1101                  0.952               0.951   
              1102                  0.881               1.052   
              1103                  1.094               1.004   
              1104                  1.025               1.054   

                                                               \
date                  2025-06-01 00:00:00 2025-05-31 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  1.119               1.016   
              1101                  1.014               0.849   
              1102                  1.040               1.008   
              1103                  1.059               1.063   
              1104                  0.995               0.891   

                                                               \
date                  2025-05-30 00:00:00 2025-05-29 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  1.054               0.982   
              1101                  0.982               0.938   
              1102                  1.089               1.139   
              1103                  0.859               0.707   
              1104                  0.882               0.751   

                                           ...                      \
date                  2025-05-28 00:00:00  ... 2025-05-10 00:00:00   
model_name    unit_no                      ...                       
アイムジャグラーEX-TP 1100                  1.118  ...               1.019   
              1101                  0.674  ...               1.081   
              1102                  0.987  ...               1.109   
              1103                  1.173  ...               0.821   
              1104                  1.140  ...               0.931   

                                                               \
date                  2025-05-09 00:00:00 2025-05-08 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  0.705               0.809   
              1101                  0.862               0.948   
              1102                  0.902               0.862   
              1103                  0.710               0.974   
              1104                  0.722               0.903   

                                                               \
date                  2025-05-07 00:00:00 2025-05-06 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  0.783               1.105   
              1101                  0.922               0.900   
              1102                  0.861               1.066   
              1103                  0.876               1.059   
              1104                  0.740               1.124   

                                                               \
date                  2025-05-05 00:00:00 2025-05-04 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  1.098           

In [ ]:
rb_rate = details["rb_rate"].copy()
rb_rate = rb_rate[~rb_rate.iloc[:, 2].isna()]
rb_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
rb_rate.to_csv(csv_path)
sheet_name = "RB_RATE_HISTORY"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
# add_spreadsheet(spreadsheet, sheet_name, rb_rate, sheet_clear=True)
rb_rate.head()

RB_RATE                                          \
date                    total 2025-06-05 00:00:00 2025-06-04 00:00:00   
model_name    unit_no                                                   
アイムジャグラーEX-TP 1100      342.8               760.2               477.2   
              1101      378.0               285.5               423.5   
              1102      375.0                 NaN               418.7   
              1103      360.6               590.9              1054.0   
              1104      337.1               257.0               411.8   

                                                               \
date                  2025-06-03 00:00:00 2025-06-02 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  299.5               376.5   
              1101                  626.0               326.8   
              1102                    NaN               296.6   
              1103                 1028.0               764.0   
              1104                  369.6               279.7   

                                                               \
date                  2025-06-01 00:00:00 2025-05-31 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  290.4               290.1   
              1101                  255.1               614.0   
              1102                  449.8               293.2   
              1103                  241.3               313.8   
              1104                  306.8                 NaN   

                                                               \
date                  2025-05-30 00:00:00 2025-05-29 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  266.8              1167.0   
              1101                    NaN               386.7   
              1102                  290.4               294.8   
              1103                  244.9               732.0   
              1104                  819.0               367.0   

                                           ...                      \
date                  2025-05-28 00:00:00  ... 2025-05-10 00:00:00   
model_name    unit_no                      ...                       
アイムジャグラーEX-TP 1100                  306.9  ...               449.3   
              1101                  280.0  ...                 NaN   
              1102                  624.0  ...               638.2   
              1103                  281.8  ...               419.0   
              1104                  605.3  ...               755.7   

                                                               \
date                  2025-05-09 00:00:00 2025-05-08 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  479.5               323.5   
              1101                  478.0              1442.0   
              1102                  464.8               881.5   
              1103                  660.0               391.0   
              1104                  411.7               398.8   

                                                               \
date                  2025-05-07 00:00:00 2025-05-06 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  708.5               326.7   
              1101                  286.1                 NaN   
              1102                  590.2               465.2   
              1103                 1390.0               309.6   
              1104                  330.2               320.2   

                                                               \
date                  2025-05-05 00:00:00 2025-05-04 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  261.7               802.7   
           

## 一日ごとの分析


In [ ]:
today = datetime.date.today()
df_rank_all = pd.DataFrame()

In [ ]:
ndays_ago = 3


def grap_culc(ndays_ago):
    base_date = today - relativedelta(days=ndays_ago)
    df_db = create_df_from_database(
        hall_name, base_date, base_date, model_name="ゴーゴージャグラー3"
    )
    df, model_list = df_preprocessing(df_db, hall_name)
    df_medals = df.pivot_table(index="unit_no", columns="date", values="medals")
    df_medals = df_medals.iloc[:-1, :]
    df_medals.reset_index(drop=True, inplace=True)
    # display(df_medals)

    base_date = today - relativedelta(days=ndays_ago + 1)
    start_date = today - relativedelta(days=ndays_ago + 7)
    df_db = create_df_from_database(
        hall_name, start_date, base_date, model_name="ゴーゴージャグラー3"
    )
    df, model_list = df_preprocessing(df_db, hall_name)
    yesterday_str = base_date.strftime("%m/%d")
    df_rank = df.pivot_table(
        index="unit_no",
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name=yesterday_str,
    )
    df_rank = df_rank.iloc[:-1, :]
    df_rank["rank"] = (
        df_rank[yesterday_str].rank(method="min", ascending=True).astype(int)
    )
    df_rank = df_rank[["rank", yesterday_str]]
    df_rank.reset_index(drop=True, inplace=True)
    # display(df_rank)

    df_merged = pd.concat([df_medals, df_rank], axis=1).sort_values("rank")
    df_merged = df_merged.iloc[:, 0]
    # display(df_merged)
    return df_merged

    # df_rank_all = pd.concat([df_rank_all.reset_index(drop=True), df_merged.reset_index(drop=True)], axis=1)
    # df_rank_all

In [ ]:
df_rank_all = pd.DataFrame()
for ndays_ago in range(1, 30):
    df_merged = grap_culc(ndays_ago)
    df_rank_all = pd.concat(
        [df_rank_all.reset_index(drop=True), df_merged.reset_index(drop=True)], axis=1
    )
df_rank_all["total"] = df_rank_all.sum(axis=1)
# df_rank_all["mean"] = df_rank_all.mean(axis=1)
c = [df_rank_all.columns[-1]] + list(df_rank_all.columns[:-1])
df_rank_all[c]

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 20 x 11
📅 検索期間: 2025-06-06 ～ 2025-06-06 📅 抽出期間: 2025-06-06 ～ 2025-06-06
含まれる日数 : 1
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 140 x 11
📅 検索期間: 2025-05-30 ～ 2025-06-05 📅 抽出期間: 2025-05-30 ～ 2025-06-05
含まれる日数 : 7
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 20 x 11
📅 検索期間: 2025-06-05 ～ 2025-06-05 📅 抽出期間: 2025-06-05 ～ 2025-06-05
含まれる日数 : 1
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。


C:\Users\mkyag\AppData\Local\Temp\ipykernel_17304\3378177172.py:4: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_rank_all = pd.concat([df_rank_all.reset_index(drop=True), df_merged.reset_index(drop=True)], axis=1)


データサイズ: 140 x 11
📅 検索期間: 2025-05-29 ～ 2025-06-04 📅 抽出期間: 2025-05-29 ～ 2025-06-04
含まれる日数 : 7
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 20 x 11
📅 検索期間: 2025-06-04 ～ 2025-06-04 📅 抽出期間: 2025-06-04 ～ 2025-06-04
含まれる日数 : 1
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 140 x 11
📅 検索期間: 2025-05-28 ～ 2025-06-03 📅 抽出期間: 2025-05-28 ～ 2025-06-03
含まれる日数 : 7
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 20 x 11
📅 検索期間: 2025-06-03 ～ 2025-06-03 📅 抽出期間: 2025-06-03 ～ 2025-06-03
含まれる日数 : 1
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 140 x 11
📅 検索期間: 2025-05-27 ～ 2025-06-02 📅 抽出期間: 2025-05-27 ～ 2025-06-02
含まれる日数 : 7
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 20 x 11
📅 検索期間: 2025-06-02 ～ 2025-06-02 📅 抽出期間: 2025-06-02 ～ 2025-06-02
含まれる日数 : 1
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 140 x 11
📅 検索期間: 2025-05-26 ～ 2025-06-01 📅 抽出期間: 2025-05-26 ～ 2025-06-01
含まれる日数 : 7
データ前処理を行います
🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 20 x 11
📅 検索期間: 2025-06-01 ～ 2025-06-01 📅 抽出期間: 20

,total,2025-06-06 00:00:00,2025-06-05 00:00:00,2025-06-04 00:00:00,2025-06-03 00:00:00,2025-06-02 00:00:00,2025-06-01 00:00:00,2025-05-31 00:00:00,2025-05-30 00:00:00,2025-05-29 00:00:00,...,2025-05-18 00:00:00,2025-05-17 00:00:00,2025-05-16 00:00:00,2025-05-15 00:00:00,2025-05-14 00:00:00,2025-05-13 00:00:00,2025-05-12 00:00:00,2025-05-11 00:00:00,2025-05-10 00:00:00,2025-05-09 00:00:00
0,-6357.0,-415.0,-2285.0,-232.0,-650.0,-182.0,-1653.0,-1900.0,-2515.0,924.0,...,-982.0,NaN,NaN,1862.0,-344.0,-444.0,-521.0,-1476.0,747.0,-503.0
1,9532.0,1241.0,-1124.0,-538.0,-332.0,NaN,NaN,NaN,556.0,NaN,...,NaN,-326.0,-1394.0,NaN,-538.0,-1218.0,-268.0,6938.0,1841.0,-226.0
2,10135.0,-794.0,-744.0,-532.0,-409.0,-1265.0,-124.0,-450.0,NaN,-338.0,...,3659.0,1088.0,394.0,3306.0,159.0,818.0,-250.0,853.0,-362.0,1826.0
3,303.0,-1194.0,2444.0,-468.0,NaN,-179.0,7459.0,-603.0,-274.0,-268.0,...,168.0,-1594.0,-903.0,1206.0,688.0,-1265.0,-1218.0,-618.0,-888.0,-185.0
4,10673.0,-415.0,-709.0,65.0,1088.0,-285.0,512.0,-803.0,1865.0,1453.0,...,-1218.0,1029.0,-256.0,2056.0,3.0,547.0,1888.0,-1759.0,-768.0,191.0
5,610.0,-309.0,-526.0,NaN,1065.0,138.0,1065.0,-156.0,-615.0,579.0,...,1476.0,1935.0,-650.0,-2109.0,-415.0,-403.0,-991.0,382.0,-1441.0,-674.0
6,8272.0,-309.0,1441.0,453.0,932.0,-403.0,-241.0,441.0,-379.0,147.0,...,-562.0,-1147.0,144.0,68.0,844.0,-203.0,1065.0,618.0,1065.0,1297.0
7,-2347.0,-115.0,347.0,-838.0,771.0,162.0,-409.0,-585.0,-874.0,-844.0,...,462.0,924.0,-9.0,2074.0,-191.0,NaN,NaN,2497.0,-91.0,-226.0
8,-6264.0,-109.0,79.0,1547.0,-185.0,-79.0,244.0,-324.0,-421.0,-938.0,...,-759.0,21.0,-356.0,429.0,NaN,38.0,-1100.0,-1747.0,-771.0,-421.0
9,-284.0,-403.0,-791.0,-709.0,-982.0,-1029.0,-1865.0,1265.0,285.0,-638.0,...,218.0,-959.0,-297.0,-194.0,-921.0,-232.0,-1441.0,2674.0,1229.0,44.0
